In [131]:
import pandas as pd 
import numpy as np
import random
import datetime
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt 
import utilities as utils
import fractionalDiff as fd
from functools import partial
import datatransformation as dtr

In [132]:
########################## Initialization ##########################
today = datetime.date.today()
bottomDate = today
bottomDate = bottomDate.replace(year = bottomDate.year - 2, month=1, day = 1)
lastYear = today.year - 1
lastYearDate = today 
lastYearDate = lastYearDate.replace(year = bottomDate.year - 1, month=1, day = 1)

selectedZones = ['NORD', 'CNOR', 'CSUD', 'CALA', 'SUD', 'SICI', 'SARD']

sqlIpex = f'SELECT * FROM [4E].[pri].[IPEX] where idzona in \'{selectedZones}\' and data>=\'{bottomDate}\''

sqlBands = 'SELECT * FROM [4E].[par].[v_Calendario]'
sqlCct = f'SELECT * FROM [4E].[pri].[IPEX] where Tipo = \'P\' and mercato = \'MGP\' and data>=\'{bottomDate}\''

ipex = dtr.ExecuteDownloadFromDb(sqlCct)
bands = dtr.ExecuteDownloadFromDb(sqlBands)[['data', 'ora', 'POP']]

In [133]:
ipex.Data.min(), ipex.Data.max()

('2020-01-01', '2022-06-17')

In [169]:
def q_k(x, k):
    return np.quantile(x, k)

defQ = partial(q_k, k = 0.99)

########################## Fixes and creates useful columns ##########################

ipex = ipex.loc[ipex['Ora']!=25]
ipex[('year')] = pd.DatetimeIndex(ipex[('Data')]).year
ipex[('quarter')] = pd.DatetimeIndex(ipex[('Data')]).quarter
ipex[('month')] = pd.DatetimeIndex(ipex[('Data')]).month
ipex[('yearMonth')] = ipex[('year')]*100 + ipex[('month')]
ipex[('yearQuarter')] = ipex[('year')]*100 + ipex[('quarter')]

########################## Creates CCT prices by zones ##########################

pun = ipex.loc[ipex['IDZona']=='PUN', ['Data', 'Ora', 'Valore']]
pzPun = pd.merge(pun, ipex, how='right', left_on=['Data','Ora'], \
    right_on=['Data', 'Ora'])
pzPun.rename(columns={'IDZona':'zone', 'Valore_x':'pun', 'Valore_y':'pz'}, inplace = True)#SS (shape storico?)
pzPun['cct'] = pzPun['pun'] - pzPun['pz']
selectedZones = ['NORD', 'CNOR', 'CSUD', 'CALA', 'SUD', 'SICI', 'SARD']
pzPun = pzPun.loc[pzPun['zone'].isin(selectedZones)]

########################## Calculates statistics for distribution of CCT ##########################

quantiles = np.linspace(0.0, 1.0, 10)
nQuantiles = len(quantiles)
functions = {'cct':[np.mean, np.std] + [partial(q_k, k = q) for q in quantiles]}

pzPunStats = pzPun.groupby(['zone', 'yearQuarter', 'yearMonth']).agg(functions)
pzPunStats = pzPunStats.droplevel(axis = 1, level = [0]).reset_index()
pzPunStats.columns = ['zone', 'yearQuarter', 'yearMonth', 'mean', 'std'] + [f'q_{k}' for k in range(0,nQuantiles)]

########################## Merge for POP calculations ##########################

pzPunBands = pd.merge(pzPun, bands, how = 'right', left_on=['Data', 'Ora'], right_on=['data', 'ora'])[['Data', 'Ora', 'POP', 'pun', 'Mercato', 'zone', 'Tipo', 'pz', 'year',
       'quarter', 'month', 'yearMonth', 'yearQuarter', 'cct']]
pzPunBands.dropna(inplace = True)
pzPunBands = pzPunBands[['Mercato', 'zone', 'year', 'yearQuarter', 'yearMonth', 'Data', 'Ora', 'POP', 'pun', 'pz', 'cct']]

monthlyPrices = pzPunBands[['zone', 'POP', 'yearMonth', 'cct']].groupby(['yearMonth', 'POP', 'zone']).agg({'cct':np.mean})
monthlyPrices = monthlyPrices.reset_index()

blPrices = pzPunBands[['zone', 'POP', 'yearMonth', 'cct']].copy()
blPrices.loc[:, 'POP'] = 'BL'
blPrices = blPrices[['zone', 'POP', 'yearMonth', 'cct']].groupby(['yearMonth', 'POP', 'zone']).agg({'cct':np.mean})
blPrices = blPrices.reset_index()
blPrices = blPrices.loc[blPrices['zone'].isin(selectedZones)]

#monthlyPrices.pivot(index = ['yearMonth', 'POP'], columns = ['zone'], values = ['cct'])

monthlyPrices = pd.concat([monthlyPrices, blPrices], axis = 0)
monthlyPrices = monthlyPrices.loc[monthlyPrices['POP'].isin(['P', 'BL'])]
monthlyPrices = monthlyPrices.loc[monthlyPrices['zone'].isin(selectedZones)]

monthlyPrices['asset'] = monthlyPrices['zone'] + '_' + monthlyPrices['POP']

monthlyPricesByZone = monthlyPrices.pivot(index = ['yearMonth'], columns = ['asset'], values = ['cct']).fillna(0)
monthlyPricesByZone = monthlyPricesByZone.droplevel(level = [0], axis = 1).reset_index()
monthlyPricesByZone = monthlyPricesByZone.loc[~monthlyPricesByZone['CALA_BL'].isna()]
monthlyPricesByZone.drop(columns = ['PUN_BL', 'PUN_P'], inplace = True, errors='ignore')
monthlyPricesByZone.rename(columns={'yearMonth':'date'}, inplace = True)
monthlyPricesByZone.head()

asset,date,CALA_BL,CALA_P,CNOR_BL,CNOR_P,CSUD_BL,CSUD_P,NORD_BL,NORD_P,SARD_BL,SARD_P,SICI_BL,SICI_P,SUD_BL,SUD_P
0,202001.0,0.0,0.0,1.011680,0.410739,-0.805856,-0.102618,1.577971,1.376464,-0.805856,-0.102618,-11.327504,-12.873443,-0.708032,0.150213
1,202002.0,0.0,0.0,0.295054,-0.425497,-0.607702,-0.501742,0.599197,-0.405247,-0.607702,-0.501742,-4.260129,1.732541,1.134396,4.323620
2,202003.0,0.0,0.0,0.368210,0.031728,-0.811409,-0.231756,0.569398,0.147544,-0.811409,-0.231756,-1.613618,-0.679159,-0.519350,0.590213
3,202004.0,0.0,0.0,0.023112,0.167413,-0.405150,0.045807,0.342159,-0.000944,0.284112,1.962428,-1.324282,-1.297343,-0.367539,0.148381
4,202005.0,0.0,0.0,-0.750991,-0.865863,-1.273728,0.034770,0.742152,-0.376051,0.482240,2.198661,-1.144748,1.954404,-0.544397,2.184962


In [170]:
returns = monthlyPricesByZone.copy()
for i in range(1, monthlyPricesByZone.shape[0]):
    returns.iloc[i, 1:monthlyPricesByZone.shape[1]] = 2 * (monthlyPricesByZone.iloc[i, 1:] - monthlyPricesByZone.iloc[i - 1, 1:] >= 0 ) - 1

returns['date'] = returns['date'].apply(lambda x: datetime.datetime.strptime(str(int(x)), '%Y%m'))
# returns.drop(columns=['yearMonth'], inplace = True)
returns.iloc[0, 1:] = 0.0
returns.head()

asset,date,CALA_BL,CALA_P,CNOR_BL,CNOR_P,CSUD_BL,CSUD_P,NORD_BL,NORD_P,SARD_BL,SARD_P,SICI_BL,SICI_P,SUD_BL,SUD_P
0,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-02-01,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0
2,2020-03-01,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0
3,2020-04-01,1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,-1.0
4,2020-05-01,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0


In [171]:
monthlyPricesByZone['date'] = monthlyPricesByZone['date'].apply(lambda x: datetime.datetime.strptime(str(int(x)), '%Y%m'))
monthlyPricesByZone.head()

asset,date,CALA_BL,CALA_P,CNOR_BL,CNOR_P,CSUD_BL,CSUD_P,NORD_BL,NORD_P,SARD_BL,SARD_P,SICI_BL,SICI_P,SUD_BL,SUD_P
0,2020-01-01,0.0,0.0,1.011680,0.410739,-0.805856,-0.102618,1.577971,1.376464,-0.805856,-0.102618,-11.327504,-12.873443,-0.708032,0.150213
1,2020-02-01,0.0,0.0,0.295054,-0.425497,-0.607702,-0.501742,0.599197,-0.405247,-0.607702,-0.501742,-4.260129,1.732541,1.134396,4.323620
2,2020-03-01,0.0,0.0,0.368210,0.031728,-0.811409,-0.231756,0.569398,0.147544,-0.811409,-0.231756,-1.613618,-0.679159,-0.519350,0.590213
3,2020-04-01,0.0,0.0,0.023112,0.167413,-0.405150,0.045807,0.342159,-0.000944,0.284112,1.962428,-1.324282,-1.297343,-0.367539,0.148381
4,2020-05-01,0.0,0.0,-0.750991,-0.865863,-1.273728,0.034770,0.742152,-0.376051,0.482240,2.198661,-1.144748,1.954404,-0.544397,2.184962


In [172]:
returns.to_excel('./data/cct_logics.xlsx', engine = 'openpyxl', index = False)
monthlyPricesByZone.to_excel('./data/cct_prices.xlsx', engine = 'openpyxl', index = False)